# 🎬 ViarteIA en Google Colab

**Stack**: T4 GPU + Groq (Llama 3) + Ngrok + Stable Video Diffusion

> ✅ Reemplaza `TU_CLAVE_DE_GROQ` y `TU_NGROK_TOKEN` antes de ejecutar

In [ ]:
# Instalar Node.js y dependencias de Python
!curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
!sudo apt-get install -y nodejs

!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install diffusers transformers accelerate safetensors
!pip install insightface basicsr facexlib
!pip install flask flask-cors python-dotenv groq
!pip install pyngrok

In [ ]:
# Clonar repositorio y configurar entorno
import os

if os.path.exists('/content/ViarteIA'):
    print("🔄 Actualizando repositorio ViarteIA...")
    %cd /content/ViarteIA
    !git pull origin main
else:
    print("⬇️ Clonando repositorio ViarteIA...")
    !git clone https://github.com/Juanpalojime/ViarteIA.git
    %cd /content/ViarteIA

!mkdir -p /content/ViarteIA/backend/python-ai/results

%%writefile backend/node-api/.env
GROQ_API_KEY=TU_CLAVE_DE_GROQ
JWT_SECRET=clave_jwt_aleatoria_de_64_caracteres_segura_1234567890
DATABASE_URL=file:./dev.db
S3_ENDPOINT=http://localhost:9000
S3_ACCESS_KEY=minioadmin
S3_SECRET_KEY=minioadmin
PYTHON_API_URL=http://localhost:8000
PORT=3001

In [ ]:
%%writefile backend/python-ai/app.py
import os
import torch
from flask import Flask, request, jsonify, send_from_directory
from groq import Groq
from diffusers import StableVideoDiffusionPipeline
from PIL import Image
import io
import base64

app = Flask(__name__)
app.config['RESULTS_FOLDER'] = '/content/ViarteIA/backend/python-ai/results'

# Inicializar Groq
groq_client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# Cargar SVD
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")

if device == "cuda":
    svd_pipe = StableVideoDiffusionPipeline.from_pretrained(
        "stabilityai/stable-video-diffusion-img2vid-xt",
        torch_dtype=torch.float16,
        variant="fp16"
    ).to(device)
    svd_pipe.enable_model_cpu_offload()
    svd_pipe.enable_xformers_memory_efficient_attention()
    print("✅ Modelo SVD cargado")
else:
    svd_pipe = None
    print("⚠️ GPU no disponible. SVD desactivado.")

# Endpoints de la API...
if __name__ == '__main__':
    app.run(host='0.0.0.0', port=8000)

In [ ]:
# Iniciar servidores
%cd /content/ViarteIA/backend/node-api

# ------------------------------------------------------------------------------
# 🔧 PARCHE AUTOMÁTICO PARA COLAB (Desactiva Prisma y Auth)
# Esto permite correr el backend sin configurar base de datos ni autenticación
# ------------------------------------------------------------------------------
import os

def patch_file(path, replacements):
    if not os.path.exists(path):
        print(f"⚠️ No se encontró: {path}")
        return
    with open(path, 'r') as f:
        content = f.read()
    for old, new in replacements.items():
        content = content.replace(old, new)
    with open(path, 'w') as f:
        f.write(content)
    print(f"✅ Parcheado: {path}")

# Parches para server.ts
patch_file('src/server.ts', {
    "import { PrismaClient } from '@prisma/client';": "// import { PrismaClient } from '@prisma/client';",
    "import { authRoutes } from './routes/auth';": "// import { authRoutes } from './routes/auth';",
    "const prisma = new PrismaClient();": "const prisma: any = { generation: { create: async (a: any) => ({ id: '123', ...a.data }), update: async () => ({}), findMany: async () => [] } }; // Mock Prisma for Colab",
    "api.register(authRoutes, { prefix: '/auth' });": "// api.register(authRoutes, { prefix: '/auth' });",
})

# Parches para generation.ts (Desactivar auth)
patch_file('src/routes/generation.ts', {
    "onRequest: [fastify.authenticate]": "// onRequest: [fastify.authenticate]",
})

# ------------------------------------------------------------------------------

!npm install --legacy-peer-deps

import subprocess
subprocess.Popen(['python', '/content/ViarteIA/backend/python-ai/app.py'])

# Este comando se quedará en ejecución
!npm run dev

In [ ]:
# Exponer con Ngrok
from pyngrok import ngrok

ngrok.set_auth_token("TU_NGROK_TOKEN")
public_url = ngrok.connect(3001)
print(f"🌍 Backend accesible en: {public_url}")